## 환경설정

In [3]:
import os
import sys
import itertools
import pickle as pk
import pandas as pd
from copy import deepcopy
from collections import defaultdict
from collections import defaultdict, Counter
from object import NaviSystem, Activity, Grid, Project
from naviutil import NaviPath, NaviFunc
navipath = NaviPath()
navifunc = NaviFunc()


__file__ = os.getcwd()

rootpath = os.path.sep.join(os.path.dirname(os.path.abspath(__file__)).split(os.path.sep)[:-1])
sys.path.append(rootpath)



# Naviutil.py

In [4]:


class NaviPath:
    root = os.path.dirname(os.path.abspath(__file__))
    
    fdir_template = os.path.sep.join((root, 'template'))
    fdir_component = os.path.sep.join((root, 'component'))
    fdir_data = os.path.sep.join((root, 'data'))
    fdir_proj = os.path.sep.join((root, 'proj'))
    fdir_schedule = os.path.sep.join((root, 'schedule'))

    activity_table = os.path.sep.join((fdir_template, 'activity_table.xlsx'))
    activity_order = os.path.sep.join((fdir_template, 'activity_order.xlsx'))
    activity_pre_dist = os.path.sep.join((fdir_template, 'activity_pre_dist.xlsx')) #choi 추가

    navisystem = os.path.sep.join((fdir_component, 'navisystem.pk'))

    def case(self, case_num):
        return os.path.sep.join((self.fdir_data, 'case_{}.xlsx'.format(case_num)))

    def proj(self, case_num):
        return os.path.sep.join((self.fdir_proj, 'proj_{}.pk'.format(case_num)))

    def schedule(self, case_num, note):
        return os.path.sep.join((self.fdir_schedule, 'schedule_N-{}_C-{}.xlsx'.format(case_num, note)))


In [5]:
class NaviFunc:
    def euclidean_distance(self, x, y):
        x_arr = np.array(x)
        y_arr = np.array(y)
        dist = np.linalg.norm(y_arr-x_arr)
        return dist

    def assign_activity_to_grid(self, schedule):
        work_plan = defaultdict(list)
        for location in schedule:
            for activity_code, day in schedule[location].items():
                work_plan[day].append((location, activity_code))

        xs, ys, zs = [], [], []
        for location in schedule:
            x, y, z = location.split('_')
            xs.append(int(x))
            ys.append(int(y))
            zs.append(int(z))

        layout_format = np.chararray((max(xs), max(ys)), itemsize=6, unicode=True)
        layout_format[:] = '------'

        work_layout = {}
        for day in work_plan:
            daily_layout = {z: deepcopy(layout_format) for z in set(zs)}
            for location, activity_code in work_plan[day]:
                x, y, z = location.split('_')
                col = int(x)-1
                row = int(y)-1
                flr = int(z)
                daily_layout[flr][row, col] = activity_code

            work_layout[day] = daily_layout
            del daily_layout

        return work_layout

    def print_schedule(self, schedule, timesleep=1):
        work_layout = self.assign_activity_to_grid(schedule)
        for day in sorted(work_layout.keys(), reverse=False):
            time.sleep(timesleep)
            print('\n\n\n\n============================================================')
            print('Work Layout: (Day: {})'.format(day))
            for flr in sorted(work_layout[day].keys(), reverse=False):
                print('--------------------------------------------------')
                print('Floor: {}'.format(flr))
                print(work_layout[day][flr])

# init.py

In [6]:
'''
A sourcecode to initialize the program.
It defines global variables and constraints.
'''

#!/usr/bin/env python
# -*- coding: utf-8 -*-

# Configuration
# import os
# rootpath = os.path.sep.join(os.path.dirname(os.path.abspath(__file__)).split(os.path.sep)[:-1])

# import pickle as pk
# import pandas as pd
# from copy import deepcopy
# from collections import defaultdict


# import sys
# sys.path.append(rootpath)
# from object import NaviSystem, Activity, Grid, Project
# from naviutil import NaviPath
# navipath = NaviPath()


def set_navisystem():
    activity_table = pd.read_excel(navipath.activity_table)
    activities = {}
    for _, line in activity_table.iterrows():
        category = line['category']
        major = line['major_activity']
        minor = line['minor_activity']
        code = line['code']
        productivity = line['productivity']

        parameters = {
            'category': category,
            'major': major,
            'minor': minor,
            'code': code,
            'productivity': productivity,
        }
        activities[code] = Activity(parameters=parameters)

    return NaviSystem(activities=activities)

def init_activity_order(navisystem):
    activity_order = pd.read_excel(navipath.activity_order)
    key_errors = []
    for _, line in activity_order.iterrows():
        predecessor_code = line['predecessor']
        successor_code = line['successor']

        try:
            predecessor_activity = deepcopy(navisystem.activities[predecessor_code])
        except KeyError:
            key_errors.append(predecessor_code)
            continue
        try:
            successor_activity = deepcopy(navisystem.activities[successor_code])
        except KeyError:
            key_errors.append(successor_code)
            continue

        predecessor_activity.add_successor(successor_activity.code)
        successor_activity.add_predecessor(predecessor_activity.code)

        navisystem.activities[predecessor_code] = predecessor_activity
        navisystem.activities[successor_code] = successor_activity

    print('============================================================')
    print('Errors on Activities')
    if key_errors:
        key_errors = list(set(key_errors))
        for code in key_errors:
            print('  | Absent in NaviSystem: {}'.format(code))
    else:
        pass

    return navisystem

def set_activity_order_recursively(navisystem):
    navisystem = init_activity_order(navisystem)

    updates = [True]
    while any(updates) == True:
        updates = []
        for activity_code in navisystem.activities:
            activity = navisystem.activities[activity_code]

            existing_preds = deepcopy(list(set(activity.predecessor)))
            for pred_code in activity.predecessor:
                pred_of_pred = list(set(navisystem.activities[pred_code].predecessor))
                updated_preds = deepcopy(list(set(existing_preds+pred_of_pred)))
                if set(updated_preds) != set(existing_preds):
                    navisystem.activities[activity_code].predecessor = updated_preds
                    updates.append(True)
            
            existing_succs = deepcopy(list(set(activity.successor)))
            for succ_code in activity.successor:
                succ_of_succ = list(set(navisystem.activities[succ_code].successor))
                updated_succs = deepcopy(list(set(existing_succs+succ_of_succ)))
                if set(updated_succs) != set(existing_succs):
                    navisystem.activities[activity_code].successor = updated_succs
                    updates.append(True)

    return navisystem

def save_navisystem(navisystem):
    os.makedirs(navipath.fdir_component, exist_ok=True)
    with open(navipath.navisystem, 'wb') as f:
        pk.dump(navisystem, f)

    print('============================================================')
    print('Save NaviSystem:')
    print('  | FilePath: {}'.format(navipath.navisystem))
    print('  | # of Activities: {}'.format(len(navisystem)))

def load_navisystem():
    with open(navipath.navisystem, 'rb') as f:
        navisystem = pk.load(f)
    return navisystem

def define_works(navisystem, case_data):
    works = defaultdict(list)
    for idx, line in case_data.iterrows():
        x = int(line['x'])
        y = int(line['y'])
        z = int(line['z'])
        location = '{}_{}_{}'.format(x, y, z)

        try:
            code = line['code']
            activity = navisystem.activities[code]
        except KeyError:
            continue

        works[location].append(activity)

    return works

def initiate_project(case_num, duration):
    navisystem = load_navisystem()

    print('============================================================')
    print('Init Project')

    case_data = pd.read_excel(navipath.case(case_num))
    works = define_works(navisystem, case_data)
    grids = []
    for loc in works:
        grids.append(Grid(location=loc, works=works[loc]))

    project = Project(activities=navisystem.activities, grids=grids, duration=duration)
    project.summary()

    return project

def save_project(case_num, project):
    os.makedirs(navipath.fdir_proj, exist_ok=True)
    with open(navipath.proj(case_num), 'wb') as f:
        pk.dump(project, f)

    print('============================================================')
    print('Save Project:')
    print('  | FilePath: {}'.format(navipath.proj(case_num)))


if __name__ == '__main__':
    ## Init NaviSystem
    navisystem = set_navisystem()
    navisystem_ordered = set_activity_order_recursively(navisystem=navisystem)
    save_navisystem(navisystem=navisystem_ordered)

    ## Project Constraints
    case_num = '05_structure'
    duration = 60
    
    ## Init Project

    project = initiate_project(case_num, duration)
    save_project(case_num, project)

Errors on Activities
  | Absent in NaviSystem: W10051
  | Absent in NaviSystem: W10041
  | Absent in NaviSystem: W10080
  | Absent in NaviSystem: W10061
Save NaviSystem:
  | FilePath: D:\cns\navi-master\navi\component\navisystem.pk
  | # of Activities: 167
Init Project
Project Summary
Save Project:
  | FilePath: D:\cns\navi-master\navi\proj\proj_05_structure.pk


# checking inputs1

In [113]:
#나옴
#print(project)
#print(project.activities.keys)
#print("""++++++++++++++++++++++++++++++++++++++++++++++++++++""")
#print(navisystem)
#print(navisystem.activities) 
#print("""++++++++++++++++++++++++++++++++++++++++++++++++++++""")
#print(project.duration)

###안나옴
#print(local_schedule) 
#print(navisystem.activity_code)
#print(navisystem.activities[activity_code])
#print(project.activity_code)
#print(project.location)
#print(navisystem.location)
#print(activity)
#print(works)
#print(navisystem.works)
#print(project.works)
#print(project.grid)


print(initiate_project('05_structure',60))
print(initiate_project.grid)
project.navisystem
project.days
navisystem.schedule
schedule.works
navisystem.grid
nums_navisystem = len(navisystem)
print(nums_navisystem)

for line_navisys in navisystem.activities:
    print(line_navisys)

for line_navisys in navisystem.activities:
    print(line_navisys)


print(project.activities)



Init Project
Project Summary


AttributeError: 'function' object has no attribute 'grid'

# update.py

In [ ]:

#!/usr/bin/env python
# -*- coding: utf-8 -*-

# Configuration
# import os
# import sys
# rootpath = os.path.sep.join(os.path.dirname(os.path.abspath(__file__)).split(os.path.sep)[:-1])
# sys.path.append(rootpath)

# from naviutil import NaviPath, NaviFunc
# navipath = NaviPath()
# navifunc = NaviFunc()

# import itertools
# import pandas as pd
# import pickle as pk
# from copy import deepcopy
# from collections import defaultdict, Counter

In [ ]:
## Project Management
def load_project(case_num):
    with open(navipath.proj(case_num), 'rb') as f:
        project = pk.load(f)

    project.export(fpath=navipath.schedule(case_num, 'initial'))
    return project


In [ ]:

def save_project(project, case_num):
    note = '{}_updated'.format(case_num)
    with open(navipath.proj(note), 'wb') as f:
        pk.dump(project, f)



In [ ]:

def sort_local_schedule(local_schedule):
    return sorted(local_schedule.items(), key=lambda x:x[1], reverse=False)




In [ ]:
def calculate_activity_work_plan(schedule):
    work_plan = defaultdict(list)
    for location in schedule:
        for activity_code, day in schedule[location].items():
            work_plan[day].append(activity_code)

    return work_plan



In [ ]:
def compare_schedule(schedule_1, schedule_2):
    if schedule_1.keys() != schedule_2.keys():
        return 'different'
    else:
        pass

    for location in schedule_1.keys():
        if len(schedule_1[location]) != len(schedule_2[location]):
            return 'different'
        else:
            pass

        for activity_code, day in schedule_1[location].items():
            if day != schedule_2[location][activity_code]:
                return 'different'
            else:
                continue

    return 'same'

In [ ]:

## Duplicated Activity Normalization
def normallize_duplicated_activity(local_schedule):
    local_schedule_modi = {}

    day = 0
    for activity_code in local_schedule.keys():
        if activity_code not in local_schedule_modi.keys():
            local_schedule_modi[activity_code] = day
            day += 1
        else:
            continue

    return local_schedule_modi






In [ ]:

## Activity Order Constraint
def check_activity_order_on_single_location(local_schedule):
    global project
    results = []

    activity_sorted_by_day = {workday: activity_code for activity_code, workday in sorted(local_schedule.items(), key=lambda x:x[1], reverse=False)}
    for activity_code_1, activity_code_2 in itertools.combinations(activity_sorted_by_day.values(), r=2):
        activity_1 = project.activities[activity_code_1]
        activity_2 = project.activities[activity_code_2]

        if activity_code_2 in activity_1.successor:
            results.append('fine')
        elif activity_code_2 in activity_1.predecessor:
            results.append('conflict')
        else:
            results.append('irrelevant')

    if any([result == 'conflict' for result in results]):
        return 'conflict'
    else:
        return 'fine'


In [ ]:

def reorder_activity_uniformly_on_single_range(local_schedule, day_from, day_to):
    for a, d in local_schedule.items():
        if d > day_from and d <= day_to:
            local_schedule[a] -= 1
    return local_schedule



In [ ]:

def push_workdays_reordering(local_schedule):
    global project

    local_schedule_modi = None
    for activity_code, workday in local_schedule.items():
        activity = project.activities[activity_code]
        later_activity_list = {a: d for a, d in local_schedule.items() if d > workday}

        target_days = []
        for later_activity_code, later_activity_day in later_activity_list.items():
            if later_activity_code in activity.predecessor:
                target_days.append(later_activity_day)

        if not target_days:
            continue
        else:
            break_point = max(target_days)
            local_schedule_modi = reorder_activity_uniformly_on_single_range(local_schedule, day_from=workday, day_to=break_point)
            local_schedule[activity_code] = break_point

    return local_schedule_modi


In [ ]:

def activity_order_constraint(schedule):
    for location in schedule:
        if check_activity_order_on_single_location(local_schedule=schedule[location]) == 'conflict':
            schedule[location] = deepcopy(push_workdays_reordering(local_schedule=schedule[location]))
        else:
            continue

    return schedule

# 작업별 순서에 정수값을 줄 수는 없을까요, 10000, 5자리 숫자 중 100의자리 이상은 선후관계 따라 부여
# 10의 자리 수는 선후관계가 없는 것들끼리 이름순 등 랜덤



In [ ]:

## Activity Productivity Constraint
def check_productivity_overload(activity_code, count):
    if count > project.activities[activity_code].productivity:
        return 'overloaded'
    else:
        return 'fine'



In [ ]:

def push_workdays_uniformly(schedule, location, after):
    schedule_to_modi = deepcopy(schedule)
    for activity_code in schedule_to_modi[location]:
        workday = schedule_to_modi[location][activity_code]
        if workday >= after:
            schedule_to_modi[location][activity_code] += 1
    return schedule_to_modi


In [ ]:

def activity_productivity_constraint(schedule, work_plan):
    global project

    for day, works in sorted(work_plan.items(), key=lambda x:x[0], reverse=False):
        activity_counter = Counter(works)
        for activity_code, count in activity_counter.items():
            if check_productivity_overload(activity_code, count) == 'overloaded':
                num_overloaded = (count-project.activities[activity_code].productivity)

                target_location_list = []
                for location in schedule.keys():
                    try:
                        if schedule[location][activity_code] == day:
                            target_location_list.append(location)
                    except KeyError:
                        continue

                for target_location in target_location_list[:num_overloaded]:
                    schedule = deepcopy(push_workdays_uniformly(schedule, location=target_location, after=day))
            else:
                continue

    return schedule


In [ ]:

# 생산성은 동일레벨(Z값이 동일 그리드끼리만)계산
# Z값이 다를 경우 다른 일정으로 진행
# 현재는 1개 팀으로 운영기준
# 전체기간 내 완료가 불가능할 경우, 팀의 수를 늘려야하며,
# 각 작업팀이 Z값이 다른 그리드에서 작업하도록 하거나 동일한 Z내에서 거리가 먼 영역을 작업하도록 함.

### Activity Predecessor Completion Constraint
# def check_activity_dis(activity_code):
#     for day, works, location in ????: #

# # import updated schedule
# 날짜, 그리드, 작업, 작업선행완료 거리값으로 디셔너리 생성
# 그리드와 작업선행완료 거리값이 해당되는 그리드값 생성
# # 작업별 순서 절대값 비교


# def check_activity_type(activity_code):
#     if activity_code[0] == "d":
#         #activiy code가 d로 시작하면 위에서 아래로, 그리드 z값이 1씩 줄어드는 순서로 작업한다
#         #선행완료 거리 제약에 1,1,-2는 *,*,-1의 거리 범위에 선행작업이 완료되어야 작업가능
#         # excavation

#         return .....
#     elif activity_code[0] == "s":
#         #activiy code가 s로 시작하면 위에서 아래로, 그리드 z값이 1씩 늘어나는 순서로 작업한다
#         #선행완료 거리 제약에 1,1,-1는 *,*,-2의 거리 범위에 선행작업이 완료되어야 작업가능
#         # structure
#         return .....
#     elif activity_code[0] == "m":
#         #activiy code가 m로 시작하면 z값을 무시 작업한다
#         # milestone
#         return .....
#     else:
#         #activiy code가 s나 d가 아니면
#         #선행완료 거리 제약에 1,1,-1는 *,*,-1의 거리 범위에 선행작업이 완료되어야 작업가능 (동일 레벨만 고려하고)
#         #동일 레벨에서 생산성을 반영
#         return 'fine'

# #작업이 선행작업

In [ ]:
## Update schedule
def update(original_schedule):

    ## Work Plans
    work_plan = calculate_activity_work_plan(original_schedule)

    ## Activity Order Constraint
    updated_schedule = deepcopy(activity_order_constraint(original_schedule))

    ## Activity Predecessor Completion Constraint
    # updated_schedule = deepcopy(activity_predecessor_completion_constraint(updated_schedule))

    ## Activity Productivity Constraint
    updated_schedule = deepcopy(activity_productivity_constraint(updated_schedule, work_plan))

    return updated_schedule



In [ ]:

def export_schedule(schedule, iteration):
    global project

    project.schedule = deepcopy(schedule)
    project.export(fpath=navipath.schedule(case_num, 'updated_I-{:03,d}'.format(iteration)))


In [ ]:

def update_schedule(project):
    print('============================================================')
    print('Update schedule')

    ## Duplicated Activity Normalization
    original_schedule = defaultdict(dict)
    for location in project.schedule:
        original_schedule[location] = deepcopy(normallize_duplicated_activity(local_schedule=project.schedule[location]))

    iteration = 0
    while True:
        print('\r  | Iteration: {:03,d}'.format(iteration), end='')
        updated_schedule = deepcopy(update(original_schedule))
        export_schedule(updated_schedule, iteration)

        if compare_schedule(original_schedule, updated_schedule) == 'same':
            break
        else:
            original_schedule = deepcopy(updated_schedule)
            iteration += 1

    project.schedule = updated_schedule
    print('\n  | Done')
    return updated_schedule


    

In [ ]:

if __name__ == '__main__':
    ## Load project
    case_num = '05_structure'
    project = load_project(case_num=case_num)

    ## Update schedule
    ############
    project.schedule = update_schedule(project)
    project.initialize()

    ## Print schedule
    navifunc.print_schedule(schedule=project.schedule)

    ## Save project
    save_project(project, case_num)

# Predecessor_dist_constraint_test

In [ ]:

# # Configuration
# import os
# import sys

# __file__ = os.getcwd()

# rootpath = os.path.sep.join(os.path.dirname(os.path.abspath(__file__)).split(os.path.sep)[:-1])
# sys.path.append(rootpath)

# from copy import deepcopy
# import pickle as pk
# import pandas as pd
# from collections import defaultdict

# from naviutil import NaviPath
# navipath = NaviPath()


def load_project(case_num):
    with open(navipath.proj(case_num), 'rb') as f:
        project = pk.load(f)

    return project




In [ ]:
def calculate_activity_work_plan(schedule):
    work_plan = defaultdict(list)
    for location in schedule:
        for activity_code, day in schedule[location].items():
            work_plan[day].append(activity_code)

    return work_plan



In [ ]:
def find_workdays(activity_list, major_code):
    workdays = []
    for activity, day in activity_list.items():
        if major_code in activity:
            workdays.append(day)

    try:
        day_start = min(workdays)
        day_end = max(workdays)
    except ValueError:
        day_start = None
        day_end = None

    return day_start, day_end


In [ ]:
def find_upper_location(location):
    x, y, z = location.split('_')
    upper_z = str(int(z) + 1)

    upper_location = '_'.join((x, y, upper_z))
    return upper_location





In [ ]:
def find_lower_location(location):
    x, y, z = location.split('_')
    lower_z = str(int(z) - 1 )

    lower_location = '_'.join((x, y, lower_z))
    return lower_location



### check inputs

In [127]:
#나옴
# print(project)
# print(project.activities.keys)
# print("""++++++++++++++++++++++++++++++++++++++++++++++++++++""")
# print(navisystem)
# print(navisystem.activities) 
# print("""++++++++++++++++++++++++++++++++++++++++++++++++++++""")
# print(project.duration)

###안나옴
# print(work_plan)
# activity_list.item()
# print(workdays)
# print(schedule) 
# print(local_schedule) 
#print(navisystem.activity_code)
#print(navisystem.activities[activity_code])
#print(project.activity_code)
#print(project.location)
#print(navisystem.location)
#print(activity)
#print(works)
#print(navisystem.works)
#print(project.works)
#print(project.grid)


#print(initiate_project('05_structure',60))
# print(initiate_project.grid)
project.navisystem
project.days
navisystem.schedule
schedule.works
navisystem.grid
nums_navisystem = len(navisystem)
print(nums_navisystem)

for line_navisys in navisystem.activities:
    print(line_navisys)

for line_navisys in navisystem.activities:
    print(line_navisys)


print(project.activities)

AttributeError: 'Project' object has no attribute 'navisystem'

#### 거리제약

In [129]:
#update schedule파일을 가져온다 
#def load_project(case_num):

#날짜와 그리드별 작업 리스트를 가져온다.

#첫 그리드(예, 2_2_1)에 배정된 작업과 작업의 선행완료 거리값을 확인한다.
#예를 들어 거리값이 3을 확인한다.

def act_pred_dist(location, activity):
     return predecessor_dist
    
#     activity_pre_dist = os.path.sep.join((fdir_template, 'activity_pre_dist.xlsx'))
#     activity_pre_dist = pd.read_excel(navipath.activity_pre_dist)
#     
#     for _, line in activity_pre_dist.iterrows(): #이해안됨 def set_navisystem(): 참조함.
#         code = line['code']
#         predecessor_dist = line['predecessor_dist']
#         parameters = {
#             'code':code
#             'predecessor_dist':predecessor_dist
#         activities[code] = Activity(parameters=parameters)
#        .....


#거리값내에 포함된 그리드 리스트를 만든다

def find_pre_dist_location_x_list(location, predecessor_dist):
    return pre_dist_location_x_list

    # 그리드 x값에 거리값을 더하고 뺀값을 리스트로 추가
    # x+3, x+2, x+1, x+0, x-1, x-2, x-3 (예, 2_2_1 =>  x=2, 5,4,3,2,1,0,-1)
    # x는 0보다 커야한다. 0보다 작으면 무시 (예, 2_2_1 =>  x=2, 5,4,3,2,1))
    # x는 그리드 최대값보다 작아야한다. 최대값보다 크면 제거 (예, 2_2_1 =>  max(x)= 4, x=4,3,2,1))
    
    x, y, z = location.split('_')
    pred_dist_value = act_pred_dist(location, activity)         #파일 activity_pre_dist.xlsx의 작업별 거리정의 값(2열)
    pre_dist_location_x_list=[]
    while pred_dist_value > 0
        pred_dist_x = int(x + pred_dist_value)
        pre_dist_location_x_list += pred_dist_x

    while pred_dist_value > 0 and x > 0
        pred_dist_x = int(x - pred_dist_value)
        pre_dist_location_x_list += pred_dist_x

    return pre_dist_location_y_list
    
def find_pre_dist_location_y_list(location, predecessor_dist):
    # 그리드 y값에 거리값을 더하고 뺀값을 리스트로 추가
    # y+3, y+2, y+1, y+0, y-1, y-2, y-3 (예, 2_2_1 =>  y=2, 5,4,3,2,1,0,-1)
    # y는 0보다 커야한다. 0보다 작으면 제거 무시 (예, 2_2_1 =>  y=2, 5,4,3,2,1))
    # y는 그리드 최대값보다 작아야한다. 최대값보다 크면 제거 (예, 2_2_1 =>  max(y)=4, y=4,3,2,1))
    
    x, y, z = location.split('_')
    pred_dist_value = act_pred_dist(location, activity)         #파일 activity_pre_dist.xlsx의 작업별 거리정의 값(2열)
    pre_dist_location_y_list=[]
    while pred_dist_value > 0 :
        pred_dist_y = int(y + pred_dist_value)
        pre_dist_location_y_list += pred_dist_y

    while pred_dist_value > 0 and y > 0:
        pred_dist_y = int(y - pred_dist_value)
        pre_dist_location_y_list += pred_dist_y
    

    return pre_dist_location_y_list

    
def find_pre_dist_location_z_list(location, activity_code):
    # 그리드 z값은 동일
    # z+0 (예, 2_2_1 =>  z=1, 1))
    # 단 Activity code가 D 인거는 z+0과 z+1값을 리스트로 가집
    # 단 Activity code가 S 인거는 z+0과 z-1값을 리스트로 가집

    x, y, z = location.split('_')
    pre_dist_location_z_list=[z]
    if activity_code in "D":
        pre_dist_location_z_list += z+1
        
    elif activity_code in "S":
        pre_dist_location_z_list += z-1
    
    else :
        pre_dist_location_z_list = z
    
    return pre_dist_location_z_list
    
    
def make_pre_dist_location_list(location, pre_dist_location_x_list, pre_dist_location_y_list, pre_dist_location_z_list):    
    for i in range(len(pre_dist_location_x_list):
            pre_dist_location_x_list[i]
        for pred_dist_y in pre_dist_location_y_list:
            for pred_dist_z in pre_dist_location_z_list:
                pred_dist_location = '_'.join(pre_dist_location_x_list[i], pre_dist_location_y_list[0], pre_dist_location_z_list[0])
    
    pred_dist_location_list += pred_dist_location
                   
    return pre_dist_location_list

    # 그리드 x 첫번째와 y값 전체, z값 결합
    # 그리드 x 두번째와 y값 전체, z값 결합
    # ...
    # 그리드 x 마지막번째와 y값 전체, z값 결합
    # 거리값내 있는 그리드 리스트를 만든다 (예, 2_2_1 => 5_5_1, 5_4_1, 5_3_1, 5_2_1, 5_1_1, 4_5_1,,... )
    # 자신의 그리드는 제외한다. 
    
    
def make_pre_dist_activity_list(updated_schedule, pre_dist_location_list):    
# 그리드 리스트에 배정되어 있는 Activity 리스트를 만든다.
    return pre_dist_activity_list

def check_pre_dist_activity_list(location, pre_dist_activity_list):    
# 그리드 리스트에 배정된 Activity 리스트와 기준 그리드의 작업과 선후비교한다.
# 특정 그리드(예, 2_2_1)에 배정된 작업과 선행작업 여부 검토
# 선행작업 있으면 작업을 민다

    if 리스트에 선행작업이 있으면 : 
        일정을 민다
    else : 
        일정을 유지한다. 


SyntaxError: invalid syntax (<ipython-input-129-93e63eaf3a19>, line 58)

### 그리드 모으기

In [ ]:
#동일한 일정에 동일한 작업은 한팀의 작업은 인접영역에서 이루어진다
#생산성 이내이지만 영역이 떨어지면 생산성이 1씩떨어진다.

# 그리드 센다
# 그리드끼리 거리를 잰다
# 생산성이 3칸이면 거리합은 

# minimize:sum((xi-xj)+ (yi-yj))
# 생산성이 3이면 4가 최적
# 4가 넘으면 생산성 -1

# 생산성이 4이면 4가 최적
# 4가 넘으면 생산성 -1

# a(1,2), b(2,2), c(1,3) = a,b(1)+a,c(1)+b,c(1+1)= 4
# a(1,2), b(2,2), c(2,3) = a,b(1)+a,c(1+1)+b,c(1)= 4
# a(1,2), b(2,2), c(3,2) = a,b(1)+a,c(2)+b,c(1)= 4


# a(1,2), b(2,2), c(4,4) = a,b(1)+a,c(3+2)+(2+2) = 6
# a(1,2), b(1,3), c(4,4) = a,b(1)+a,c(3+2)+(3+1) = 10
# 생산성 2로 조정?



In [ ]:
def push_workdays_uniformly(schedule, location, after, add):
    schedule_to_modi = deepcopy(schedule)
    for activity_code in schedule_to_modi[location]:
        workday = schedule_to_modi[location][activity_code]
        if workday >= after:
            schedule_to_modi[location][activity_code] += add
    return schedule_to_modi



In [ ]:
def compare_schedule(schedule_1, schedule_2):
    if schedule_1.keys() != schedule_2.keys():
        return 'different'
    else:
        pass

    for location in schedule_1.keys():
        if len(schedule_1[location]) != len(schedule_2[location]):
            return 'different'
        else:
            pass

        for activity_code, day in schedule_1[location].items():
            if day != schedule_2[location][activity_code]:
                return 'different'
            else:
                continue

    return 'same'



In [ ]:

if __name__ == '__main__':
    ## Load project
    case_num = '03_excavation_only'
    project = load_project(case_num=case_num)

    fname = 'schedule_N-03_excavation_only_C-updated_I-005.xlsx'
    fdir = 'd:/cns/navi-master/navi/schedule'
    fpath = os.path.join(fdir, fname)
    schedule_data = pd.read_excel(fpath)

    schedule = defaultdict(dict)
    for row in schedule_data.iterrows():
        # print(row)
        location = row[1]['Unnamed: 0']
        schedule[location] = {}

        for day, activity_code in row[1].items():
            if day == 'Unnamed: 0':
                continue

            if len(str(activity_code)) == 6:
                schedule[location][activity_code] = day


    iteration = 0
    while True:


        for location in schedule.keys():
            my_start, my_end = find_workdays(activity_list=schedule[location], major_code='D')
            # print(location)
            # print(my_start)
            # print(my_end)
            print('-----------------------------------')

            upper_location = find_upper_location(location)
            if upper_location not in schedule.keys():
                continue
            # print(location)
            # print(upper_location)

            upper_start, upper_end = find_workdays(activity_list=schedule[upper_location], major_code='D')

            try:
                if my_start < upper_end:
                    print(location)
                    print(my_start)
                    print(upper_location)
                    print(upper_end)

                    add = upper_end - my_start + 1
                    print(add)
                    updated_schedule = deepcopy(push_workdays_uniformly(schedule, location, after=my_start, add=add))
                    iteration += 1

                else:
                    continue
            except TypeError:
                continue

        if compare_schedule(schedule, updated_schedule) == 'same':
            break
        else:
            schedule = deepcopy(updated_schedule)
            iteration += 1

    print(iteration)

    schedule_dict = defaultdict(dict)
    for location in updated_schedule:
        for activity_code, day in updated_schedule[location].items():
            schedule_dict[day][location] = activity_code

    schedule_df = pd.DataFrame(schedule_dict)
    schedule_df.to_excel('temp.xlsx', na_rep='', header=True, index=True)